# PYTHON Y SQLite en Jupyter Notebook

En este ejemplo interactivo, se va a utilizar Python con Jupiter Notebook, para realizar conexion y operaciones con una base de datos SQLite, utilizando la Base de datos de ejemplo sakila.bd el cual se encuentra libre para descargar del siguiente link: https://github.com/bradleygrant/sakila-sqlite3.git


<img src="sakila.png">

# 1. Alistamos el entorno de configuracion

Importamos las librerias de **sqlite3** para manejar la BD SQLite, **Pandas** para usar su data frame, **pprint** prety print para imprimir las listas de forma adecuada, **os** para poder ejecutar comandos del sistema

In [15]:
import sqlite3
import pandas as pd
from pprint import pprint
import os

## 2. Cargamos la base de datos

El archivo de base de datos debe estar junto con el Notebook, a partir de esta ruta se conecta con la base de datos, se debe recordar que SQLite es una base de datos local que se almacena en un archivo

In [16]:
#sea cuidadoso como escribe el path y el nombre que apunta a la BD
path=os.getcwd() #como extraer el working directory para apuntar a la BD
path=path+'\\sakila_master.db' #completamos con el nombre de la bd
sqliteConnection=sqlite3.connect(path)
cursor=sqliteConnection.cursor()

## 3. Realizamos Query con sqlite3

Realizada la conexion, en la tabla de sistema sqlite_master, listamos las tablas existentes en la BD y mostramos el contenido.

In [78]:
cursor.execute("SELECT count(*) FROM sqlite_master") #verificamos que existen 91 elementos
pprint(cursor.fetchall())

[(92,)]


In [79]:
#resultado=cursor.execute("SELECT name FROM sqlite_master;") #se enlistan los 91 elementos
#pprint(resultado.fetchall())

In [80]:
#se agrupa por tipo los elementos en la base de datos y se cuentan, agrupados por tipo

query="SELECT type,count(type) as cantidad FROM sqlite_master GROUP BY type"
salida=cursor.execute(query)
pprint(salida.fetchall())
#se tiene 40 index, 18 table, 30 trigger, 5 view

[('index', 40), ('table', 17), ('trigger', 30), ('view', 5)]


In [81]:
resultado=cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #se enlistan las tablas 17
pprint(resultado.fetchall())

[('actor',),
 ('country',),
 ('city',),
 ('address',),
 ('language',),
 ('category',),
 ('customer',),
 ('film',),
 ('film_actor',),
 ('film_category',),
 ('film_text',),
 ('inventory',),
 ('staff',),
 ('store',),
 ('payment',),
 ('rental',),
 ('sqlite_sequence',)]


## 3.1. Se realiza codigo para identificacion de tablas

In [83]:
#requiero describir la estructura de las tablas indicada.
tabla='city' #escriba el nombre de una tabla

query="PRAGMA table_info("+tabla+");" #se solicita informacion de la estructura de la tabla
salida=cursor.execute(query)
print("La estructura de la tabla: ",tabla)
print("----------------------------------------------")
pprint(salida.fetchall())

query="SELECT count(*) from "+tabla #se solicita informacion de la cantidad de elementos de la tabla
salida=cursor.execute(query)
print("----------------------------------------------")
print("La cantidad de datos en la tabla es: ",salida.fetchall())


La estructura de la tabla:  city
----------------------------------------------
[(0, 'city_id', 'INT', 1, None, 1),
 (1, 'city', 'VARCHAR(50)', 1, None, 0),
 (2, 'country_id', 'SMALLINT', 1, None, 0),
 (3, 'last_update', 'TIMESTAMP', 1, None, 0)]
----------------------------------------------
La cantidad de datos en la tabla es:  [(600,)]


In [84]:
#considerando la tabla anterior, se seleccionan los primeros 10 elementos de dicha tabla
query="SELECT * FROM "+tabla+ " LIMIT 500"
salida=cursor.execute(query)
pprint(salida.fetchall())


[(1, 'A Corua (La Corua)', 87, '2020-12-23 07:12:14'),
 (2, 'Abha', 82, '2020-12-23 07:12:14'),
 (3, 'Abu Dhabi', 101, '2020-12-23 07:12:14'),
 (4, 'Acua', 60, '2020-12-23 07:12:14'),
 (5, 'Adana', 97, '2020-12-23 07:12:14'),
 (6, 'Addis Abeba', 31, '2020-12-23 07:12:14'),
 (7, 'Aden', 107, '2020-12-23 07:12:14'),
 (8, 'Adoni', 44, '2020-12-23 07:12:14'),
 (9, 'Ahmadnagar', 44, '2020-12-23 07:12:14'),
 (10, 'Akishima', 50, '2020-12-23 07:12:14'),
 (11, 'Akron', 103, '2020-12-23 07:12:14'),
 (12, 'al-Ayn', 101, '2020-12-23 07:12:14'),
 (13, 'al-Hawiya', 82, '2020-12-23 07:12:14'),
 (14, 'al-Manama', 11, '2020-12-23 07:12:14'),
 (15, 'al-Qadarif', 89, '2020-12-23 07:12:14'),
 (16, 'al-Qatif', 82, '2020-12-23 07:12:14'),
 (17, 'Alessandria', 49, '2020-12-23 07:12:14'),
 (18, 'Allappuzha (Alleppey)', 44, '2020-12-23 07:12:14'),
 (19, 'Allende', 60, '2020-12-23 07:12:14'),
 (20, 'Almirante Brown', 6, '2020-12-23 07:12:14'),
 (21, 'Alvorada', 15, '2020-12-23 07:12:14'),
 (22, 'Ambattur', 44,

In [91]:
#SE REALIZA CODIGO PARA OBTENER UNA TABLA QUE DESCRIBA A TODAS LAS TABLAS EXISTENTES DENTRO DE LA BD SAKILA
def obtener_informacion_tablas():
    # Obtener la lista de tablas en la base de datos
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cursor.fetchall()

    # Imprimir encabezados del cuadro
    print("{:<20} {:<20} {:<20}".format("Nombre de Tabla", "Cantidad de Registros", "Cantidad de Columnas"))
    print("="*60)

    for tabla in tablas:
        nombre_tabla = tabla[0]

        # Obtener la cantidad de registros
        cursor.execute(f"SELECT COUNT(*) FROM {nombre_tabla};")
        cantidad_registros = cursor.fetchone()[0]

        # Obtener la cantidad de columnas
        cursor.execute(f"PRAGMA table_info({nombre_tabla});")
        cantidad_columnas = len(cursor.fetchall())

        # Imprimir la información de la tabla
        print("{:<20} {:<20} {:<20}".format(nombre_tabla, cantidad_registros, cantidad_columnas))


# Llamar a la función para obtener la información y mostrarla
obtener_informacion_tablas()


Nombre de Tabla      Cantidad de Registros Cantidad de Columnas
actor                200                  4                   
country              109                  3                   
city                 600                  4                   
address              603                  8                   
language             6                    3                   
category             16                   3                   
customer             599                  9                   
film                 1000                 13                  
film_actor           5462                 3                   
film_category        1000                 3                   
film_text            0                    3                   
inventory            4581                 4                   
staff                2                    11                  
store                2                    4                   
payment              16049                7           

## 3.2. Se realiza codigo creacion de tablas

In [90]:
#se solicita la creacion de una tabla llamada archivos.

query="""CREATE TABLE IF NOT EXISTS archivos (  
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NOMBRE TEXT COLLATE NOCASE,
    TIPO TEXT COLLATE NOCASE,
    TAMANO INTEGER UNSIGNED,
    FECHAHORA TIMESTAMP,
    RUTA TEXT COLLATE NOCASE,
    UBICA TEXT COLLATE NOCASE,
    COMENTA TEXT COLLATE NOCASE,
    CATEGORIA TEXT COLLATE NOCASE,
    PROYECTO TEXT COLLATE NOCASE,
    FAV INTEGER
);"""
salida=cursor.execute(query)
print(salida.fetchall())

[]


In [88]:
#se solicita el borrado de la tabla llamada archivos
query="drop table archivos"
salida=cursor.execute(query)
print(salida.fetchall())

[]


## 4. Realizamos Query con dataframe de Pandas

In [8]:
query='SELECT * FROM film LIMIT 100'
df=pd.read_sql_query(query,sqliteConnection, index_col="film_id")
df

,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
film_id,,,,,,,,,,,,
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2020-12-23 07:12:31
2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2020-12-23 07:12:31
3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2020-12-23 07:12:31
4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2020-12-23 07:12:31
5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2020-12-23 07:12:31
...,...,...,...,...,...,...,...,...,...,...,...,...
96,BREAKING HOME,A Beautiful Display of a Secret Agent And a Mo...,2006,1,None,4,2.99,169,21.99,PG-13,"Trailers,Commentaries",2020-12-23 07:12:33
97,BRIDE INTRIGUE,A Epic Tale of a Robot And a Monkey who must V...,2006,1,None,7,0.99,56,24.99,G,"Trailers,Commentaries,Behind the Scenes",2020-12-23 07:12:33
98,BRIGHT ENCOUNTERS,A Fateful Yarn of a Lumberjack And a Feminist ...,2006,1,None,4,4.99,73,12.99,PG-13,Trailers,2020-12-23 07:12:33


In [9]:
query='SELECT rating, AVG(length) FROM film GROUP BY rating HAVING AVG (length) BETWEEN 112 AND 120'
df1=pd.read_sql_query(query,sqliteConnection)
print(df1)

  rating  AVG(length)
0  NC-17   113.228571
1     PG   112.005155
2      R   118.661538


## 5. Cerrar conecciones

In [10]:
#cerrar cursor
cursor.close()
#cerrar BD
sqliteConnection.close()